In [1]:
import os
from pathlib import Path

import qcodes
from qcodes.dataset import (
    LinSweep,
    TogetherSweep,
    do1d,
    do2d,
    dond,
    Measurement,
    load_or_create_experiment,
    plot_dataset,
    initialise_or_create_database_at
)

import numpy as np

import scipy

import time

import matplotlib.pyplot as plt

## Measuring Current Pre-Amp Bias

In [2]:
# Load in Station
station = qcodes.Station(config_file='../station_config.yml')
station.load_instrument('sim900')
station.load_instrument('agilent')

Connected to: Stanford_Research_Systems SIM900 (serial:s/n130900, firmware:ver3.6) in 1.84s
Connected to: HEWLETT-PACKARD 34401A (serial:0, firmware:11-5-3) in 0.06s


<Agilent34401A: agilent>

In [ ]:
preamp_bias_exp = load_or_create_experiment('Current PreAmp-2 Bias Experiment', sample_name='10MOhm Resistor')

In [ ]:
VI_sweep = Measurement(exp=preamp_bias_exp)

In [ ]:
# Record independent variable
VI_sweep.register_parameter(station.sim900.volt_1)
# Record dependent variable (setpoints implies dependency)
VI_sweep.register_parameter(station.agilent.volt, setpoints=[station.sim900.volt_1])

dV = 1e-3 # min resolution
volt_min, volt_max = -10e-3, 10e-3
voltage_sweep = np.arange(volt_min, volt_max, dV) # sweep values
station.sim900.set_smooth({1: volt_min}) # smoothly ramp to start point

with VI_sweep.run() as datasaver:
    for voltage in voltage_sweep:
        station.sim900.volt_1.set(voltage) # set voltage (mV)
        datasaver.add_result(
            (station.sim900.volt_1, station.sim900.volt_1()),
            (station.agilent.volt, station.agilent.volt())
            ) # record measurement
        
station.sim900.set_smooth({1: 0}) # smoothly ramp to 0V after measurement
dataset = datasaver.dataset

In [ ]:
df = dataset.to_pandas_dataframe().reset_index()

df_current = df.copy()
df_current = df_current.rename(columns={'agilent_volt': 'agilent_current'})
df_current.iloc[:,1] = df_current.iloc[:,1].mul(-1e-6) # sensitivity

V = df_current['sim900_volt_1'] # V
I = df_current['agilent_current'] # A

def linear(x, m, b):
    return m * x + b

fit_params, pcov = scipy.optimize.curve_fit(linear, xdata=V, ydata=I, p0=(1, 0))

G, IBias = fit_params

R = 1/G

print("Resistance is {:3f} M\u03A9".format(R/1e6))
print("Y-Intercept is (0 mV, {:3f} nA)".format(IBias/1e-9))
print("X-Intercept is ({:3f} mV, 0 nA)".format((-IBias/G) / 1e-3))

axes = df_current.plot.scatter(y='agilent_current', x='sim900_volt_1', linewidth=1, s=20, label=r'$I_{\text{RAW}}$')
axes.axhline(y=0, color='k', alpha=0.25, linestyle='--')
axes.axvline(x=0, color='k', alpha=0.25, linestyle='--')
axes.scatter([0],[IBias], s=45, marker='x', c='r', label='Current Bias: {:.3f} nA'.format(IBias/1e-9))
axes.set_ylabel(r'$I$ (A)')
axes.set_xlabel(r'$V_{\text{SIM900}}$ (V)')
axes.set_title('CSG Current PreAmp-2 Bias')

df_current_nobias = df_current.copy()
# subtract the bias
df_current_nobias.iloc[:,1] = df_current_nobias.iloc[:,1].subtract(IBias)
axes.scatter(df_current_nobias['sim900_volt_1'], df_current_nobias['agilent_current'], color='k', s=20, label=r'$I_{\text{CORRECTED}}$')
axes.legend(loc='best')

## 0. Set-up the Dataset

In [ ]:
def exp_fit(x, a, b, x0, y0):
    return a * np.exp(b * (x-x0)) + y0

def sigmoid_fit(x, a, b, x0, y0):
    return a/(1+np.exp(b * (x-x0))) + y0

In [ ]:
initialization_db_path = os.path.join(os.getcwd(), 'initialization.db')
initialise_or_create_database_at(initialization_db_path)
initialization_exp = load_or_create_experiment('Initialization Experiment', sample_name='SHT')

In [3]:
station.sim900.set_smooth({
    'S': 0,
    'STL': 0,
    'LB': 0,
    'P': 0,
    'RB': 0,
    'STR':0
})
station.sim900.print_readable_snapshot()

sim900:
	parameter      value
--------------------------------------------------------------------------------
IDN             :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM900', ...
IDN_7           :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_8           :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_LB          :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_P           :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_RB          :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_S           :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_STL         :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
IDN_STR         :	{'vendor': 'Stanford_Research_Systems', 'model': 'SIM928', ...
smooth_timestep :	1 (s)
timeout         :	10 (s)
volt_7          :	0 (V)
volt_7_step     :	0.005 (V)
volt_8          :	0 (V)
volt_8_step     :	0.005 (V)
volt_LB

## 1. Global Turn On

In [ ]:
# Bias the device
sensitivity = 1e-6 # A/V
VBias = 10e-3 # V

station.sim900.set_smooth({'S': VBias})

time.sleep(1)

def fit_data():
    dataset =  None #get last dataset?
    
    df = dataset.to_pandas_dataframe().reset_index()
    print(df)
    df_current = df.copy()
    df_current = df_current.rename(columns={'agilent_volt': 'agilent_current'})
    df_current.iloc[:,-1] = df_current.iloc[:,-1].mul(-1e-6).subtract(IBias) # sensitivity

    axes = df_current.plot.scatter(y='agilent_current', x='sim900_volt_STL', linewidth=1, s=20, label=r'$I$')
    axes.axhline(y=0, color='k', alpha=0.25, linestyle='--')
    axes.axvline(x=0, color='k', alpha=0.25, linestyle='--')
    axes.set_ylabel(r'$I$ (A)')
    axes.set_xlabel(r'$V_{\text{All Gates}}$ (V)')
    axes.set_title('Global Turn-On')

    mask = df_current['agilent_current'] > 1e-11 # arbitrary current value, change if needed
    X_thres = df_current['sim900_volt_STL'][mask]
    Y_thres = df_current['agilent_current'][mask]
    guess = (-max(Y_thres), -1, min(X_thres), max(Y_thres))
    fit_params, fit_cov = scipy.optimize.curve_fit(exp_fit, X_thres, Y_thres, guess)
    a, b, x0, y0 = fit_params

    plt.plot(X_thres, exp_fit(X_thres, a,b,x0,y0), 'r--', label='Fit')

    print(f"Fit function: {exp_fit.__name__}")
    print(f"Fit parameter(s): a = {round(a,3)} A, b = {round(b,3)} 1/mV, (x0,y0)=({round(x0,3)} mV),{round(y0,3)} A)")

def max_current():
    I = sensitivity * station.agilent.volt()

    I_threshold = 1e-9
    if np.abs(I) > I_threshold:
        # Device has saturated
        return True

station.sim900.set_smooth({'STL': 0,
                           'LB': 0,
                           'P': 0,
                           'RB': 0,
                           'STR':0})

time.sleep(1)

max_V = 10e-3
N = int((max_V / 1e-3) + 1)

sweep_STL = LinSweep(station.sim900.volt_STL, 0, max_V, N, 0.01, get_after_set=True)
sweep_LB = LinSweep(station.sim900.volt_LB, 0, max_V, N, 0.01, get_after_set=True)
sweep_P = LinSweep(station.sim900.volt_P, 0, max_V, N, 0.01, get_after_set=True)
sweep_RB = LinSweep(station.sim900.volt_RB, 0, max_V, N, 0.01, get_after_set=True)
sweep_STR = LinSweep(station.sim900.volt_STR, 0, max_V, N, 0.01, get_after_set=True)

gate_sweeps = [sweep_STL, sweep_LB, sweep_P, sweep_RB, sweep_STR]

global_turn_on,_,_ = dond(
    TogetherSweep(
        *gate_sweeps
    ), 
    station.agilent.volt, 
    # show_progress=True, 
    # use_threads=False,
    break_condition=max_current,
    exit_actions=[fit_data],
    measurement_name='Global Turn On'
    exp=initialization_exp,
) # Return A tuple of QCoDeS DataSet, Matplotlib axis, Matplotlib colorbar.




## 2. Individual Pinch-Offs

In [ ]:
sweep_STL = LinSweep(station.sim900.volt_STL, max_V, 0, N, 0.01, get_after_set=True)
sweep_LB = LinSweep(station.sim900.volt_LB, max_V, 0, N, 0.01, get_after_set=True)
sweep_P = LinSweep(station.sim900.volt_P, max_V, 0, N, 0.01, get_after_set=True)
sweep_RB = LinSweep(station.sim900.volt_RB, max_V, 0, N, 0.01, get_after_set=True)
sweep_STR = LinSweep(station.sim900.volt_STR, max_V, 0, N, 0.01, get_after_set=True)

gate_sweeps = [
    sweep_STL,
    sweep_LB, 
    sweep_P, 
    sweep_RB,
    sweep_STR
]

def fit_data():

    dataset =  None #get last dataset?
    
    df = dataset.to_pandas_dataframe().reset_index()
    print(df)
    df_current = df.copy()
    df_current = df_current.rename(columns={'agilent_volt': 'agilent_current'})
    df_current.iloc[:,-1] = df_current.iloc[:,-1].mul(-1e-6).subtract(IBias) # sensitivity

    axes = df_current.plot.scatter(y='agilent_current', x='sim900_volt_STL', linewidth=1, s=20, label=r'$I$')
    axes.axhline(y=0, color='k', alpha=0.25, linestyle='--')
    axes.axvline(x=0, color='k', alpha=0.25, linestyle='--')
    axes.set_ylabel(r'$I$ (A)')
    axes.set_xlabel(r'$V_{\text{All Gates}}$ (V)')
    axes.set_title('Global Turn-On')

    X = df_current['sim900_volt_STL']
    Y = df_current['agilent_current']
    guess = (1,0.2,1,0)
    fit_params, fit_cov = scipy.optimize.curve_fit(sigmoid_fit, X, Y, guess)
    a, b, x0, y0 = fit_params

    plt.plot(X, sigmoid_fit(Y, a,b,x0,y0), 'r--', label='Fit')

    print(f"Fit function: {sigmoid_fit.__name__}")
    print(f"Width of sigmoid: {round(np.sqrt(8)/b,3)} mV")

def min_current():
    I = sensitivity * station.agilent.volt()

    I_threshold = 1e-12# A
    if np.abs(I) < I_threshold:
        # Device has pinched-off
        return True
    
import tqdm
for sweep in tqdm.tqdm((gate_sweeps)):
    pinch_off = dond(
        sweep,
        station.agilent.volt, 
        # do_plot=True, 
        # show_progress=True, 
        # use_threads=False,
        break_condition=min_current,
        # exit_actions=fit_data,
        measurement_name='{} Pinch Off'.format(str(sweep._param).split('_')[-1]),
        exp=initialization_exp
    )
    station.sim900.set_smooth({str(sweep._param).split('_')[-1]: max_V})
    

## 3. Barrier Barrier Sweep

In [ ]:
pinch_off_V = 0e-3
turn_on_V = 0e-3

station.sim900.set_smooth({'volt_STL': pinch_off_V})
station.sim900.set_smooth({'volt_P': pinch_off_V})
station.sim900.set_smooth({'volt_STR': pinch_off_V})

sweep_LB = LinSweep(station.sim900.volt_LB, pinch_off_V, turn_on_V, N, 0.01, get_after_set=True)
sweep_RB = LinSweep(station.sim900.volt_RB, pinch_off_V, turn_on_V, N, 0.01, get_after_set=True)

BB_sweep = dond(
    sweep_LB,
    sweep_RB,
    station.agilent.volt, 
    # do_plot=True, 
    # show_progress=True, 
    # use_threads=True,
    measurement_name='Barrier Barrier Sweep',
    exp=initialization_exp
)

## 4. Coulomb Diamonds

## 5. Reset Device Voltages

In [ ]:
station.sim900.set_smooth({
    'S': 0,
    'STL': 0,
    'LB': 0,
    'P': 0,
    'RB': 0,
    'STR':0
})